In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import skimage

import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import spams
import numpy as np
import skimage 
import tensorflow as tf
import math as m
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import layers
from mxnet import nd
import mxnet as mx
from tensorflow import keras
from tensorflow.keras import losses
import pandas as pd

2023-06-17 23:09:52.486560: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [3]:
import pickle
import pandas as pd
# 加载数据''
with open('/home/xipeng/FLcode/Organa/my_data_0.5.pkl', 'rb') as f:
    data_x , data_y,x_test,y_test  = pickle.load(f)
x_train =  data_x 
y_train =  data_y
num_classes = 11
for i in range(30):
    df = pd.DataFrame(data_y[i])
    label_counts = df.sum(axis=0)
    label_counts = label_counts[label_counts > 0]
# 提取标签和数量
    labels = np.array(label_counts.index, dtype=np.uint8)
    counts = np.array(label_counts.values)
# 打印结果
    print(f" {i}: ",(labels, counts))

 0:  (array([3, 4, 6, 7, 8], dtype=uint8), array([ 97., 238., 353., 236., 228.], dtype=float32))
 1:  (array([ 2,  3,  4,  7, 10], dtype=uint8), array([111., 105., 315., 325., 296.], dtype=float32))
 2:  (array([2, 3, 4, 7, 8], dtype=uint8), array([103., 106., 298., 337., 308.], dtype=float32))
 3:  (array([1, 2, 3, 5, 6], dtype=uint8), array([111., 116., 116., 309., 500.], dtype=float32))
 4:  (array([1, 2, 3, 5, 9], dtype=uint8), array([123., 155., 160., 418., 296.], dtype=float32))
 5:  (array([ 1,  3,  7,  8, 10], dtype=uint8), array([106., 120., 330., 313., 283.], dtype=float32))
 6:  (array([ 0,  6,  7,  8, 10], dtype=uint8), array([113., 384., 239., 220., 196.], dtype=float32))
 7:  (array([0, 2, 6, 7, 9], dtype=uint8), array([141., 118., 402., 264., 227.], dtype=float32))
 8:  (array([0, 1, 2, 4, 9], dtype=uint8), array([185., 130., 138., 375., 324.], dtype=float32))
 9:  (array([ 1,  4,  7,  8, 10], dtype=uint8), array([ 97., 274., 256., 280., 245.], dtype=float32))
 10:  (arr

In [4]:
#对前百分之10的数据进行噪声处理
#for i in range(6000):
    
   # x_train[i,:,:] = skimage.util.random_noise(x_train[i,:,:],mode="gaussian",var=0.1,clip=True)

In [5]:
def segmentation_data(x_train,y_train,n):   
    data_x = {}
    data_y = {}
    t = int(60000/n)
    print(t)
    print(n)
    for i in range(n):
        data_x[i] = x_train[t*i:t+t*i,:,:]

        data_y[i] = y_train[t*i:t+t*i,:]

    print(data_x[n-1].shape)
    print(data_y[n-1].shape)
    return data_x,data_y

In [6]:
def  set_model(n):
    model_d = {}
    for i in range(n):
        model_d[i]=Sequential()
        # 定义顺序模型
        model = model_d[i]

        model.add(Convolution2D(
            input_shape = (28,28,1),
            filters = 8,
            kernel_size = 3,
            strides = 1,
            padding = 'same',
            activation = 'relu',
        ))
    # 第一个池化层
        model.add(MaxPooling2D(
            pool_size = 2,
            strides = 2,
            padding = 'same',
        ))
    # 第二个卷积层
        model.add(Convolution2D(16,3,strides=1,padding='same',activation='relu'))
    # 第二个池化层
        model.add(MaxPooling2D(2,2,'same'))
    # 把第二个池化层的输出扁平化为1维
        model.add(Flatten())
    # 第一个全连接层
        model.add(Dense(256,activation='relu'))
    # Dropout
        model.add(Dropout(0.5))
    # 第二个全连接层
        model.add(Dense(11,activation='softmax'))

    # 定义优化器
        adam = Adam(lr=5e-4)

    # 定义优化器,loss function,训练过程中计算准确率
        model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

        #print(model_d[i])
    return model_d

In [7]:
# def 定义函数 、
#return 返回函数
#想要使用函数中定义的值，想要 变量名= 函数值


In [8]:
#所有的客户端训练一遍
def fit_allmodel(model_d):
    weights_1 = {}
    for i in range(len(model_d)):
    # 训练模型
        model_d[i].fit(data_x[i],data_y[i],batch_size=64,epochs=1,verbose=0)
        loss,accuracy = model_d[i].evaluate(x_test,y_test,verbose=0)
        
        #print('test loss',loss)
        #print('test accuracy',accuracy)
        
    for i in range(len(model_d)): 
        weights_1[i] = model_d[i].get_weights() 
    return model_d , weights_1

In [9]:
def compute_median_params(models):
    """
    计算一系列模型参数的中位数。

    参数:
        models: 一系列训练好的模型。

    返回值:
        中位数参数列表。
    """
    params = [model.get_weights() for model in models]
    median_params = [np.median([param[i] for param in params], axis=0) for i in range(len(params[0]))]
    return median_params

    

In [10]:
#服务器将客户端数据集中并分配给客户端
def fed_avg(model_d):
    weights = {}
    loss_all=[]
    acc_all=[]
    data_save={}
    #这个for是得到所有模型的参数，那在这里保存所有模型的参数
    for i in range(len(model_d)): 
        weights[i] = model_d[i].get_weights() 
#################################################        
#对前5个数据进行        
    for i in range(10):
        for j in range(8):
            weights[i][j] = skimage.util.random_noise(weights[i][j],mode="gaussian",var = 0.2,clip=True)
        
        
#################################################
#保存50个变量 

####这里把形状print出来，然后修改形状。
    M_weight = mdeian_weight(weights)
    M_weight[0] = np.reshape(M_weight[0], (5, 5, 1, 30))
    M_weight[1] = np.reshape(M_weight[1], (30,))
    M_weight[2] = np.reshape(M_weight[2], (5, 5, 30, 5))
    M_weight[3] = np.reshape(M_weight[3], (5,))
    M_weight[4] = np.reshape(M_weight[4], (245, 100))
    M_weight[5] = np.reshape(M_weight[5],(100,))
    M_weight[6] = np.reshape(M_weight[6], (100, 10))
    M_weight[7] = np.reshape(M_weight[7], (10, ))
    #将集合后的数据分配给每一个客户端
    for i in range (50):
        model_d[i].set_weights(M_weight)
    
    #测试客户端的精度
    loss,accuracy= model_d[i].evaluate(x_test,y_test)
#################################################   
    return model_d,loss,accuracy
 #################################################

In [11]:
def mdeian_weight(weights):
    weight_median = weights[0].copy()
    
    for t in range (8):  
        tensors = []
        for i in range(len(weights)):
            tensors.append(weights[i][t])
        stacked_tensors = np.stack(tensors)
# Alternatively, you can use a list comprehension
# tensors = [tensor[i] for i in range(num_tensors)]
        weight_median[t] = np.median(stacked_tensors, axis=0)
# Stack the tensors along a new axis (axis 0 by default)
    return weight_median

In [12]:
def noise(weights,t):
    for i in range(t):
        for j in range(8):
            weights[i][j] = skimage.util.random_noise(weights[i][j],mode="gaussian",var = 0.2,clip=True)
    return weights

In [13]:
def avg_set(model_d,avg_weight):
    #这个for是得到所有模型的参数，那在这里保存所有模型的参数
    for i in range(len(model_d)): 
        model_d[i].set_weights(avg_weight) 
    loss,accuracy= model_d[0].evaluate(x_test,y_test)
    return model_d

In [14]:
from keras.models import load_model
from keras.models import clone_model

In [15]:

n = int(30)
model_d = set_model(n)
loss_fed = [0]*100
accuracy_fed = [0]*100

t= 9;
for s in range (100):
    model_d,weights_1 = fit_allmodel(model_d)
    weights_1 = noise(weights_1,t)
    avg_weight =   mdeian_weight(weights_1)
    model_d = avg_set(model_d,avg_weight)
    model_d[0].save('1N9.h5')
    print(s)
# data_finall 太大，不能保存， 只能分开保存，想一下 如何一次运行能保存N个文件，试一下 程序名 +n 来进行保存

2023-06-17 23:09:58.483921: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-06-17 23:09:58.519725: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-06-17 23:09:58.519788: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-06-17 23:09:58.519862: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-06-17 23:09:58.519897: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-06-17 23:09:58.519931: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

556/556 [==============================] - 2s 4ms/step - loss: 2.3973 - accuracy: 0.1138
0
556/556 [==============================] - 2s 4ms/step - loss: 2.3463 - accuracy: 0.1161
1
556/556 [==============================] - 2s 3ms/step - loss: 2.3878 - accuracy: 0.1226
2
556/556 [==============================] - 2s 4ms/step - loss: 2.2546 - accuracy: 0.3107
3
556/556 [==============================] - 2s 4ms/step - loss: 2.1969 - accuracy: 0.2083
4
556/556 [==============================] - 2s 4ms/step - loss: 2.0837 - accuracy: 0.2622
5
556/556 [==============================] - 2s 4ms/step - loss: 2.0325 - accuracy: 0.2909
6
556/556 [==============================] - 2s 4ms/step - loss: 1.9559 - accuracy: 0.3169
7
556/556 [==============================] - 2s 4ms/step - loss: 1.9162 - accuracy: 0.3462
8
556/556 [==============================] - 2s 4ms/step - loss: 1.8574 - accuracy: 0.3586
9
556/556 [==============================] - 2s 4ms/step - loss: 1.8054 - accuracy: 0.4006
1

In [16]:

n = int(30)
model_d = set_model(n)
loss_fed = [0]*100
accuracy_fed = [0]*100

t=6;
for s in range (100):
    model_d,weights_1 = fit_allmodel(model_d)
    weights_1 = noise(weights_1,t)
    avg_weight =   mdeian_weight(weights_1)
    model_d = avg_set(model_d,avg_weight)
    print(s)
    model_d[0].save('1N6.h5')
# data_finall 太大，不能保存， 只能分开保存，想一下 如何一次运行能保存N个文件，试一下 程序名 +n 来进行保存

556/556 [==============================] - 2s 4ms/step - loss: 2.3978 - accuracy: 0.1182
0
556/556 [==============================] - 2s 4ms/step - loss: 2.3678 - accuracy: 0.1794
1
556/556 [==============================] - 2s 4ms/step - loss: 2.3824 - accuracy: 0.1161
2
556/556 [==============================] - 2s 4ms/step - loss: 2.3831 - accuracy: 0.1161
3
556/556 [==============================] - 2s 4ms/step - loss: 2.2820 - accuracy: 0.2094
4
556/556 [==============================] - 2s 4ms/step - loss: 2.2203 - accuracy: 0.2401
5
556/556 [==============================] - 2s 4ms/step - loss: 2.1548 - accuracy: 0.2555
6
556/556 [==============================] - 2s 4ms/step - loss: 2.0420 - accuracy: 0.2917
7
556/556 [==============================] - 2s 4ms/step - loss: 1.9652 - accuracy: 0.3368
8
556/556 [==============================] - 2s 3ms/step - loss: 1.9020 - accuracy: 0.3703
9
556/556 [==============================] - 2s 4ms/step - loss: 1.8653 - accuracy: 0.3720
1

In [17]:

n = int(30)
model_d = set_model(n)
loss_fed = [0]*100
accuracy_fed = [0]*100

t=3;
for s in range (100):
    model_d,weights_1 = fit_allmodel(model_d)
    weights_1 = noise(weights_1,t)
    avg_weight =   mdeian_weight(weights_1)
    model_d = avg_set(model_d,avg_weight)
    print(s)
    model_d[0].save('1N3.h5')
# data_finall 太大，不能保存， 只能分开保存，想一下 如何一次运行能保存N个文件，试一下 程序名 +n 来进行保存

556/556 [==============================] - 2s 4ms/step - loss: 2.3986 - accuracy: 0.0429
0
556/556 [==============================] - 2s 4ms/step - loss: 2.3692 - accuracy: 0.1848
1
556/556 [==============================] - 2s 4ms/step - loss: 2.3710 - accuracy: 0.1848
2
556/556 [==============================] - 2s 4ms/step - loss: 2.3620 - accuracy: 0.1880
3
556/556 [==============================] - 2s 4ms/step - loss: 2.2560 - accuracy: 0.2910
4
556/556 [==============================] - 2s 4ms/step - loss: 2.1939 - accuracy: 0.1812
5
556/556 [==============================] - 2s 4ms/step - loss: 2.1090 - accuracy: 0.2502
6
556/556 [==============================] - 2s 4ms/step - loss: 2.0044 - accuracy: 0.3021
7
556/556 [==============================] - 2s 4ms/step - loss: 1.9259 - accuracy: 0.3465
8
556/556 [==============================] - 2s 4ms/step - loss: 1.8585 - accuracy: 0.3533
9
556/556 [==============================] - 2s 4ms/step - loss: 1.8017 - accuracy: 0.3742
1